In [ ]:
# import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_react_agent, AgentExecutor

In [2]:
# load env variables
load_dotenv()

True

In [14]:
llm = ChatOpenAI(model='o4-mini-2025-04-16')

In [70]:
# Create a list with the DuckDuckGo search tool
tools = DuckDuckGoSearchRun(
        name="Web Search",
        description="Useful for searching information on the internet. Use this when you need to find current or factual information.",
        verbose = True
    )

In [58]:
from langchain.memory import ConversationBufferMemory

In [59]:
# Define the prompt template for the agent
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=PromptTemplate.from_template(template)
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=ConversationBufferMemory(memory_key="chat_history")
)

/var/folders/77/4fw_96792q73dm48k4zk7_p80000gn/T/ipykernel_6229/167193580.py:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history")


In [ ]:
def ask_question(query):
    """
    Function to ask a question to the agent and get a response
    """
    try:
        response = agent_executor.invoke({"input": query})
        # response = tools.invoke(query)
        # response = agent_executor.invoke("")
        return response.get("output", "Sorry, I couldn't generate a response.")
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [ ]:
query = "how many hydrogen fuel stations in the bay area"
print(ask_question(query))

# limitations of using duckduckgosearchrun and an agent executor:
# whole query doesn't seem to be understood. the model seems sometimes focused on certain keywords at the beginning
# results from the search doesn't seem to be coming from a search engine

/Users/cindy/Documents/llm/web-search/.venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


The meaning of MANY is consisting of or amounting to a large but indefinite number. How to use many in a sentence. Find 347 different ways to say MANY, along with antonyms, related words, and example sentences at Thesaurus.com. MANY definition: 1. used mainly in negative sentences and questions and with "too", "so", and "as" to mean "a large…. Learn more. You use many to indicate that you are talking about a large number of people or things. I don't think many people would argue with that. Not many films are made in Finland. Do you keep many … 1. a. a large number of: many coaches; many times. b. (as pronoun; functioning as plural): many are seated already. 2. (foll by: a, an, or another, and a singular noun) each of a considerable number …An error occurred: 'str' object has no attribute 'get'


# Using a prompt template

In [76]:
def load_prompt_template(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [77]:
# prompt = load_prompt_template('prompt.md')

In [78]:
# Create PromptTemplate
react_prompt = PromptTemplate.from_template(prompt)

# Create agent
agent = create_react_agent(llm, tools, react_prompt)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


TypeError: expected str, got ChatPromptTemplate

In [ ]:
# Create a prompt template for main query
prompt = ChatPromptTemplate.from_template(
    "Answer the following question using the most up-to-date information you can find: {question}"
)

In [ ]:
# Create RunnableSequence
sequence = RunnableSequence(
    prompt,
    lambda prompt_result: agent_executor.invoke({
        "input": prompt_result.to_messages()[0].content,
        "tools": tools,
        "tools_names": ", ".join([tool.name for tool in tools]),
        "agent_scratchpad": ""
    })
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser


# Use sequence
result = sequence.invoke({
    "question": "mayflower dublin restaurant"
})

# the answers here are limited to factual info
print("Answer: ")
print(result)

/Users/cindy/Documents/llm/web-search/.venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Answer: 
Mayflower long distance moving customers are serviced by movers at over 500 locations, and we can help ease the burden of moving any distance, long or short. From full service moving to … At Mayflower, we aim to simplify the moving quote process so that you can make the right decision for your move and your budget. We understand that customers want to review … Mayflower’s long-distance movers are experienced tackling long-distance moves of any size. Whether you are moving a small apartment or need to move employees of a corporation to a … Experience stress-free moves with Mayflower® in Colorado Springs, CO. As the nation's trusted moving company since 1927, we offer customizable services crafted to your needs, ensuring a … Mayflower is a trusted full service moving company. Our full service movers will help you no matter where you live. Request a quote today.
